<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/TFMA_with_TF_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.0.0
!pip install  apache_beam==2.16.0
!pip install tensorflow-model-analysis==0.15.1

In [2]:
import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))

import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.version.VERSION_STRING))

TF version: 2.0.0
Beam version: 2.16.0
TFMA version: 0.15.1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/IMDB Dataset.csv', nrows=10000)
df.sentiment = df.sentiment.map({ 'positive' : 1, 'negative' : 0})
#df.loc[df['sentiment'] == 'positive', sentiment] = 1
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(df["review"].values, df['sentiment'].values, random_state=42)

Using TensorFlow backend.


In [0]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

tokenizer = Tokenizer(num_words=max_features)
trainingData = tokenizer.fit_on_texts(texts_train)

train_data = tokenizer.texts_to_sequences(texts_train)
test_data = tokenizer.texts_to_sequences(texts_test)

padded_train_data = pad_sequences(train_data, maxlen=80, padding="post")
padded_test_data = pad_sequences(test_data, maxlen=80, padding="post")

In [10]:
batch_size = 64
max_features = len(tokenizer.word_index) + 1

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 128))
model.add(tf.keras.layers.LSTM(128, dropout=0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print (model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="binary_crossentropy", metrics=['accuracy'])

model.fit(padded_train_data, y_train, epochs=2, batch_size=batch_size)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         6727552   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 6,859,265
Trainable params: 6,859,265
Non-trainable params: 0
_________________________________________________________________
None
Train on 7500 samples
Epoch 1/2
7500/7500 [==============================] - 35s 5ms/sample - loss: 0.5646 - accuracy: 0.6900
Epoch 2/2
7500/7500 [==============================] - 32s 4ms/sample - loss: 0.2933 - accuracy: 0.8852


In [11]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(padded_test_data, y_test, batch_size=64)
print('test loss, test acc:', results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(padded_test_data[:3])
print('predictions shape:', predictions.shape)


# Evaluate on test data
2500/1 [=======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [0]:
!mkdir /content/output_path

In [12]:
!mkdir latest_saved_model
model.save("/content/latest_saved_model")

mkdir: cannot create directory ‘latest_saved_model’: File exists
INFO:tensorflow:Assets written to: /content/latest_saved_model/assets


In [0]:
#review_list = tf.train.BytesList(value=texts_train)
review_list = []
sentiment_list = []

for x in range(0, len(padded_train_data)):
    review_list.append(tf.train.Feature(int64_list=tf.train.Int64List(value=padded_train_data[x])))
    sentiment_list.append(tf.train.Feature(int64_list=tf.train.Int64List(value=[y_train[x]])))

reviews = tf.train.FeatureList(feature=review_list)
sentiments = tf.train.FeatureList(feature=sentiment_list)

movie_reviews = tf.train.FeatureLists(feature_list={
    'Reviews': reviews,
    'Sentiments': sentiments
})

# Create the SequenceExample
example = tf.train.SequenceExample(feature_lists=movie_reviews)
with tf.io.TFRecordWriter('reviews.tfrecord') as writer:
    writer.write(example.SerializeToString())

In [25]:
type(example)

tensorflow.core.example.example_pb2.SequenceExample

In [15]:
!saved_model_cli show --dir "/content/latest_saved_model/"

The given SavedModel contains the following tag-sets:
serve


In [17]:
!saved_model_cli show --dir "/content/latest_saved_model/" --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [18]:
!saved_model_cli show --dir "/content/latest_saved_model/" --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['embedding_1_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1)
      name: serving_default_embedding_1_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [0]:
def run_and_render(eval_model=None, slice_list=None, slice_idx=0):
  """Runs the model analysis and renders the slicing metrics

  Args:
      eval_model: An instance of tf.saved_model saved with evaluation data
      slice_list: A list of tfma.slicer.SingleSliceSpec giving the slices
      slice_idx: An integer index into slice_list specifying the slice to use

  Returns:
      A SlicingMetricsViewer object if in Jupyter notebook; None if in Colab.
  """
  eval_result = tfma.run_model_analysis(eval_shared_model=eval_model,
                                          data_location="reviews.tfrecord",
                                          file_format='tfrecords',
                                          slice_spec=slice_list,
                                          output_path='/content/output_path',
                                          extractors=None)
  return tfma.view.render_slicing_metrics(eval_result, slicing_spec=slice_list[slice_idx])

In [26]:
import os
eval_model_dir_0 = os.path.join("/content/","latest_saved_model")
eval_shared_model_0 = tfma.default_eval_shared_model(eval_saved_model_path=eval_model_dir_0)
slices = [tfma.slicer.SingleSliceSpec(columns=['Reviews'])]

run_and_render(eval_model=eval_shared_model_0, slice_list=slices, slice_idx=0)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


RuntimeError: ignored